In [30]:
import numpy as np
import pandas as pd

from bokeh.layouts import column
from bokeh.models import CustomJS, Slider, LinearColorMapper, ColorBar
from bokeh.plotting import ColumnDataSource, figure,output_file, save, show


In [31]:
HTML_COLOR = "#CFD8DC"

In [32]:




def prediction_contour_xxformat(c_range, r_range, tempt, npoints=100) -> np.array:

    #arrays with predictors
    c = np.linspace(c_range[0], c_range[1], npoints)
    r = np.linspace(r_range[0], r_range[1], npoints)

    #2D arrays with predictors
    tt= tempt*np.ones(shape=(npoints, npoints))
    cc, rr = np.meshgrid(c, r)


    conduct_2d = -5.11*cc - 0.04*tt - 0.35*(cc**(2.5))*(rr**(0.5)) + 0.000273*(tt**(2))*(cc**(0.25))

    return c, r, conduct_2d

In [33]:
concentr, ratios, predictions2d = prediction_contour_xxformat(c_range=(0.2,3), r_range=(0,1), tempt=273.15)
predictions2d

array([[ 1.67342538e+00,  1.98684947e+00,  2.25950048e+00, ...,
         7.12609402e-01,  6.31944039e-01,  5.50824458e-01],
       [ 1.67279613e+00,  1.98597361e+00,  2.25832762e+00, ...,
         1.89748306e-01,  9.64321326e-02,  2.47984215e-03],
       [ 1.67253548e+00,  1.98561082e+00,  2.25784180e+00, ...,
        -2.68278516e-02, -1.25384162e-01, -2.24651935e-01],
       ...,
       [ 1.66722796e+00,  1.97822327e+00,  2.24794913e+00, ...,
        -4.43697519e+00, -4.64223658e+00, -4.84974369e+00],
       [ 1.66719609e+00,  1.97817892e+00,  2.24788974e+00, ...,
        -4.46345137e+00, -4.66935337e+00, -4.87751029e+00],
       [ 1.66716439e+00,  1.97813480e+00,  2.24783065e+00, ...,
        -4.48979282e+00, -4.69633216e+00, -4.90513559e+00]])

In [34]:
def prediction_contour_plot(c_range = (0.2,3) , r_range = (0,1), tempt = 273.15):

    # data 
    # must give a vector of image data for image parameter
    concentr, ratios, predictions2d = prediction_contour_xxformat(c_range=c_range, 
                                                                r_range=r_range, 
                                                                tempt=tempt, npoints=100)
    
    

    source = ColumnDataSource(data={'image': [predictions2d.tolist()], 
                                    'x': concentr.tolist(), 
                                    'y': ratios.tolist(), 
                                    'dw':[np.subtract(*c_range)],
                                    'dh':[np.subtract(*r_range)]})

    #colors
    color_mapper = LinearColorMapper(palette='Viridis256', low=np.amin(predictions2d), high=np.amax(predictions2d))  

    print(color_mapper.__dir__)

    cbar = ColorBar(color_mapper = color_mapper, 
                        title='Conductivity / mS/cm', 
                        title_text_font_size='18px',
                        major_label_text_font_size='18px', 
                        background_fill_color = HTML_COLOR)



    p = figure(plot_width=600,plot_height=400,
                x_range=c_range, y_range=r_range, 
                tooltips=[("k [mS/cm]: ", "@image")],
                border_fill_color = HTML_COLOR)


    p.image(image="image", x='x', y='y', source=source, color_mapper=color_mapper, level="image")


    #sliders and interactivity
    tempt_slider = Slider(start=-30, end=60, value=0, step=1, title="Temperature [°C]")

    callback = CustomJS(args=dict(source=source, tempt=tempt_slider, color_bar = cbar, pobj=p),
                        code="""

                        const data = source.data;
                        const image = data['image'][0];
                        const t = tempt.value + 273.15;
                        const x = data['x'];
                        const y = data['y'];

                        const b1 = -5.11;
                        const b2 = -0.04;
                        const b3 = -0.35; 
                        const b4 = 2.73e-4;

                        let max = 0;
                        let min = 100;
                       
                        for (var i = 0; i < x.length; i++) {
                            for (var j = 0; j < y.length; j++) {
                                
                                image[j][i] = b1*x[i] + b2*t + b3*(x[i]**(2.5))*(y[j]**(0.5)) + b4*(x[i]**(0.25))*(t**(2));
                                
                                if (image[j][i] > max) {
                                    max = image[j][i];
                                };

                                if (image[j][i] < min) {                                    
                                    min = image[j][i];
                                };
                            };
                        };

                        if (min<0) {
                            min = 0;
                        };

                        color_bar.color_mapper.low = min;
                        color_bar.color_mapper.high = max;
                        source.change.emit();
                                """)


    tempt_slider.js_on_change('value', callback)



    p.grid.grid_line_width = 0.5
    p.xaxis.axis_label = 'Salt concentration / mol/kg'
    p.yaxis.axis_label = 'PC:EC ratio'
    p.xaxis.major_label_text_font_size = '18px'
    p.xaxis.axis_label_text_font_size = '20px'
    p.yaxis.major_label_text_font_size = '18px'
    p.yaxis.axis_label_text_font_size = '20px'
    p.add_layout(cbar, 'right')

    layout = column(tempt_slider,p)


    return layout



    

In [35]:
output_file('./static/standalone_app.html')
layout = prediction_contour_plot()
show(layout)
save(layout)

<built-in method __dir__ of LinearColorMapper object at 0x0000019C0221D640>


'c:\\Users\\eibfl\\Nextcloud\\documents\\WP11_ML\\Electrolytes\\SR-electrolytes\\docs\\static\\standalone_app.html'